<a href="https://colab.research.google.com/github/RefineX/Change-Detection/blob/main/AML_Noisy_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Data Downloader
dataset = "WHU" #@param ["S2Looking", "LEVIRCD_Plus", "WHU"]
resize = "512" #@param ["1024", "512", "256"]
crop = "256" #@param ["1024", "512", "256"]

data_link_dict = {
    'S2Looking': {
      '1024_512': "1-0_vcODMYmyYIY_uhMcs97aAk2OJVduq&confirm=t",
      '1024_256': "1-0mVd6BjKnG3LXhYkbyxdx0-dtRKY6KO&confirm=t",
      '512_512': "1-6WOmE0LTJ4Z31EphmA0L8QY5NMMllsT&confirm=t",
      '512_256': "1-7NSOmHTsDpkEtbgt_N4VSgipCo5kr3a&confirm=t",
      '256_256': "1-8x5d5DrNsgJ5eAOYyKwi91g8knGeRjQ&confirm=t",
      '1024_1024': "1GzrgMwJKguXSWSfFsBSC2qSr52fVEc7W&confirm=t"
    },
    'LEVIRCD_Plus': {
      '1024_512': "1-22GjfF8mlLFNyoJaa6_GMJRfTCoGZyc&confirm=t",
      '1024_256': "1-2r-zCCfQjLRtwwXMGPYAxEJcTS8Hp80&confirm=t",
      '512_512': "1-43scZrxe3Q_PH2EnBRuc_6l9O9WjPs2&confirm=t",
      '512_256': "1-5oC0xV36S4K5VMiQuwheWoyMt1ymYb9&confirm=t",
      '256_256': "1-69cdgqlcXunt5vrCR9jRvcMkpPNzYWr&confirm=t",
      '1024_1024': "1nyPJZGGOL7o8A0m0rGw7wyu2BFIxC4SD&confirm=t"
    },
    'WHU': {
      '1024_512': "1-22GjfF8mlLFNyoJaa6_GMJRfTCoGZyc&confirm=t",
      '1024_256': "1aLl-squfjxvQhpqs0Y7Xz8eMwC2Vrttw&confirm=t",
      '512_512': "1-43scZrxe3Q_PH2EnBRuc_6l9O9WjPs2&confirm=t",
      '512_256': "10ui3sAPdOmCcrJO5cVjFEakMf5W8lwt6&confirm=t",
      '256_256': "1xFRwJJvLjpUsEVQVHkxCulP4Ts92OABM&confirm=t",
      '1024_1024': "1nyPJZGGOL7o8A0m0rGw7wyu2BFIxC4SD&confirm=t"
    }
}

import os
if os.path.exists(f'Data/{dataset}/{resize}_{crop}'):
  print('This dataset already exists.')
else:
  gdown_link = data_link_dict[dataset][f'{resize}_{crop}']
  !gdown "{gdown_link}"
  print('Unzipping...',end='')
  !unzip -q "{resize}_{crop}.zip"
  print('Done.\nDeleting zip...',end='')
  !rm "{resize}_{crop}.zip"
  print('Done.')

Downloading...
From: https://drive.google.com/uc?id=10ui3sAPdOmCcrJO5cVjFEakMf5W8lwt6&confirm=t
To: /content/512_256.zip
100% 27.2M/27.2M [00:00<00:00, 141MB/s] 
Unzipping...Done.
Deleting zip...Done.


In [3]:
# Import libraries
import tensorflow as tf
# import os
# import time
import numpy as np
# from tqdm.notebook import tqdm
from pathlib import Path
from matplotlib import pyplot as plt
# from IPython import display
import cv2
import pywt
from tensorflow.keras.models import load_model

In [4]:
# Dataset Characteristics
dataset = "WHU" #@param ["S2Looking", "LEVIRCD_Plus", "WHU"]
resized_size = "512" #@param [1024, 512, 256]
crop_size = "256" #@param [1024, 512, 256]
resized_size = int(resized_size)
crop_size = int(crop_size)
if dataset == 'S2Looking':
  pre_image = 'Image1'
  post_image = 'Image2'
else:
  pre_image = 'A'
  post_image = 'B'
label_image = 'label'
PATH = f"/content/Data/{dataset}/{resized_size}_{crop_size}"
Path('saved/models/').mkdir(parents=True, exist_ok=True)
Path('saved/histories/').mkdir(parents=True, exist_ok=True)

In [5]:
# Metrics

def iou(y_true,y_pred):
  y_pred = tf.argmax(y_pred,-1)
  y_true = tf.argmax(y_true,-1)
  tp = tf.cast(tf.reduce_sum(y_pred*y_true,axis=[1,2]), 'float32')
  fp = tf.cast(tf.reduce_sum(y_pred*(1-y_true),axis=[1,2]), 'float32')
  fn = tf.cast(tf.reduce_sum((1-y_pred)*y_true,axis=[1,2]), 'float32')
  iou_value = (tp + 1e-14) / (tp + fp + fn + 1e-14)
  return tf.reduce_mean(iou_value)

def miou(y_true,y_pred):
  y_pred = tf.argmax(y_pred,-1)
  y_true = tf.argmax(y_true,-1)
  tp = tf.cast(tf.reduce_sum(y_pred*y_true,axis=[1,2]), 'float32')
  tn = tf.cast(tf.reduce_sum((1-y_pred)*(1-y_true),axis=[1,2]), 'float32')
  fp = tf.cast(tf.reduce_sum(y_pred*(1-y_true),axis=[1,2]), 'float32')
  fn = tf.cast(tf.reduce_sum((1-y_pred)*y_true,axis=[1,2]), 'float32')
  iou1 = (tp + 1e-14) / (tp + fp + fn + 1e-14)
  iou2 = (tn + 1e-14) / (tn + fp + fn + 1e-14)
  iou_value = (iou1 + iou2) / 2
  return tf.reduce_mean(iou_value)

# Losses

In [6]:
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D, BatchNormalization, Activation
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import plot_model
# from classification_models.tfkeras import Classifiers
# import pickle
# from tensorflow.keras import mixed_precision
# mixed_precision.set_global_policy('mixed_float16')

In [7]:
# epochs = 50
# es_patience = 15
# rLR_patience = 5
# lr = 1e-2
# optimizer = Adam(learning_rate = lr)
# loss = 'categorical_crossentropy'
# BATCH_SIZE = 16
# metrics = [iou, 'accuracy']

## Noisy Test

In [8]:
#@title Default title text
data_augmentation = "none" #@param ["none", "edge_augmented", "mra_augmented"]

In [ ]:
# Helper Functions for Model #1

def salt_and_pepper_noise(image):
  noisy_image = image
  uniform_noise = tf.random.uniform((crop_size, crop_size),0,1)
  uniform_noise = tf.stack([uniform_noise]*noisy_image.shape[-1],axis=-1)
  # uniform_noise = tf.random.uniform(shape=image[..., -1:].shape)
  power = 0.025
  # noisy_image[uniform_noise < power] = 0
  noisy_image = tf.where(uniform_noise < power, tf.zeros_like(noisy_image), noisy_image)
  noisy_image = tf.where(uniform_noise > (1-power), tf.ones_like(noisy_image), noisy_image)
  return noisy_image

def gaussian_noise(image):
  noisy_image = image
  power = 0.2
  noisy_image = noisy_image + (tf.random.normal(noisy_image.shape) * power)
  ni_max = tf.reduce_max(noisy_image)
  ni_min = tf.reduce_min(noisy_image)
  noisy_image = (noisy_image - ni_min) / (ni_max - ni_min)
  return noisy_image

# Edge Augmentation
def canny_edge(image):
  l,u = 100, 200
  edgesA = cv2.Canny(image[:,:,:3].numpy()[:,:,::-1], l, u)
  edgesB = cv2.Canny(image[:,:,3:].numpy()[:,:,::-1], l, u)
  edges = tf.stack([edgesA,edgesB],axis=-1)
  edges = tf.cast(edges,tf.float32)
  edges /= tf.reduce_max(edges)
  return edges

# MRA Augmentation
def mra_haar(image):

  decomp = pywt.wavedec2(cv2.cvtColor(image[:,:,:3].numpy(),cv2.COLOR_RGB2GRAY),wavelet='haar',mode='constant',level=1)
  h_dA = decomp[1][0]
  h_dA = (h_dA-h_dA.min())/(h_dA.max()-h_dA.min())
  h_dA = cv2.resize(h_dA, (crop_size,crop_size), cv2.INTER_LINEAR)
  v_dA = decomp[1][1]
  v_dA = (v_dA-v_dA.min())/(v_dA.max()-v_dA.min())
  v_dA = cv2.resize(v_dA, (crop_size,crop_size), cv2.INTER_LINEAR)
  d_dA = decomp[1][2]
  d_dA = (d_dA-d_dA.min())/(d_dA.max()-d_dA.min())
  d_dA = cv2.resize(d_dA, (crop_size,crop_size), cv2.INTER_LINEAR)

  decomp = pywt.wavedec2(cv2.cvtColor(image[:,:,3:].numpy(),cv2.COLOR_RGB2GRAY),wavelet='haar',mode='constant',level=1)
  h_dB = decomp[1][0]
  h_dB = (h_dB-h_dB.min())/(h_dB.max()-h_dB.min())
  h_dB = cv2.resize(h_dB, (crop_size,crop_size), cv2.INTER_LINEAR)
  v_dB = decomp[1][1]
  v_dB = (v_dB-v_dB.min())/(v_dB.max()-v_dB.min())
  v_dB = cv2.resize(v_dB, (crop_size,crop_size), cv2.INTER_LINEAR)
  d_dB = decomp[1][2]
  d_dB = (d_dB-d_dB.min())/(d_dB.max()-d_dB.min())
  d_dB = cv2.resize(d_dB, (crop_size,crop_size), cv2.INTER_LINEAR)
  edges = tf.stack([h_dA, v_dA, d_dA, h_dB, v_dB, d_dB],axis=-1)
  edges = tf.cast(edges,tf.float32)

  return edges

def load(imageA_file):
  imageA = tf.io.read_file(imageA_file)
  imageA = tf.image.decode_png(imageA)[:,:,:3]

  imageB_file = tf.strings.regex_replace(imageA_file, f'/{pre_image}/', f'/{post_image}/')
  imageB = tf.io.read_file(imageB_file)
  imageB = tf.image.decode_jpeg(imageB)[:,:,:3]

  label_file = tf.strings.regex_replace(imageA_file, f'/{pre_image}/', f'/{label_image}/')
  label_file = tf.strings.regex_replace(label_file, '.jpg', '.png')
  label = tf.io.read_file(label_file)
  label = tf.image.decode_png(label)[:,:,0]

  imageA = tf.cast(imageA,tf.float32)
  imageB = tf.cast(imageB,tf.float32)
  image = tf.concat([imageA,imageB],axis=-1)
  label = tf.cast(label,tf.float32)
  label = tf.stack([255-label,label],axis=-1)

  return image, label

def normalize(image, label):
  image = image / 255
  label = label / 255
  return image, label

def load_image(image_file):
  image, label = load(image_file)
  image, label = normalize(image, label)
  noisy_image = tf.py_function(gaussian_noise, [image], tf.float32)
  noisy_image = tf.cast(noisy_image * 255,tf.uint8)
  if data_augmentation == 'edge_augmented':
    edges = tf.py_function(canny_edge, [noisy_image], tf.float32)
  elif data_augmentation == 'mra_augmented':
    edges = tf.py_function(mra_haar, [noisy_image], tf.float32)
  noisy_image = tf.cast(noisy_image / 255,tf.float32)
  if data_augmentation != 'none':
    noisy_image = tf.concat([noisy_image,edges],axis=-1)
  return noisy_image, label

BATCH_SIZE = 16

test_dataset = tf.data.Dataset.list_files(f'{PATH}/test/{pre_image}/*.jpg')
test_dataset = test_dataset.map(load_image)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset_length = len(test_dataset)

print(f'test_dataset_length: {test_dataset_length}')

test_dataset_length: 87


In [ ]:
!gdown "1eOeFpU2Fl4Ekn0ytqjMLVP4CuMP8xl-M&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1eOeFpU2Fl4Ekn0ytqjMLVP4CuMP8xl-M&confirm=t
To: /content/LEVIRCD_Plus_resnet18_model_ef_mra_haar_0.5603.zip
100% 156M/156M [00:02<00:00, 59.8MB/s]


In [ ]:
!unzip LEVIRCD_Plus_resnet18_model_ef_mra_haar_0.5603.zip

In [ ]:
modelpath = '/content/saved/models/LEVIRCD_Plus_resnet18_model_ef_canny1_0.5714.h5'

In [ ]:
model = load_model(modelpath, custom_objects={'iou':iou})
model.evaluate(test_dataset)

In [ ]:
"1QwvuA0Bn0wwiJ3ZnWOYsQw7PJfLQCFWh&confirm=t"

In [ ]:
modelpath = '/content/saved/models/LEVIRCD_Plus_resnet18_model_ef_mra_haar_0.5603.h5'

In [ ]:
# # Dataset test - unaugmented
# for x,y in test_dataset.take(1):
#   pass
# idx = 0
# plt.figure(figsize=(3*10,10))
# plt.subplot(131)
# plt.imshow(x[idx,:,:,:3].numpy())
# plt.subplot(132)
# plt.imshow(x[idx,:,:,3:6].numpy())
# plt.subplot(133)
# plt.imshow(y[idx].numpy().argmax(-1),cmap='gray')

In [ ]:
# # Dataset test - edge augmented
# for x,y in test_dataset.take(1):
#   pass
# idx = 0
# plt.figure(figsize=(20,4))
# plt.subplot(151)
# plt.imshow(x[idx,:,:,:3].numpy())
# plt.subplot(152)
# plt.imshow(x[idx,:,:,6].numpy(),cmap='gray')
# plt.subplot(153)
# plt.imshow(x[idx,:,:,3:6].numpy())
# plt.subplot(154)
# plt.imshow(x[idx,:,:,7].numpy(),cmap='gray')
# plt.subplot(155)
# plt.imshow(y[idx].numpy().argmax(-1),cmap='gray')

In [ ]:
# # Dataset test - mra augmented
# for x,y in test_dataset.take(1):
#   pass
# idx = 5
# plt.figure(figsize=(40,16))
# plt.subplot(251)
# plt.imshow(x[idx,:,:,:3].numpy())
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(252)
# plt.imshow(x[idx,:,:,6].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(256)
# plt.imshow(x[idx,:,:,7].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(257)
# plt.imshow(x[idx,:,:,8].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(253)
# plt.imshow(x[idx,:,:,3:6].numpy())
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(254)
# plt.imshow(x[idx,:,:,9].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(258)
# plt.imshow(x[idx,:,:,10].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(259)
# plt.imshow(x[idx,:,:,11].numpy(),cmap='gray')
# plt.axis('off')
# plt.tight_layout()
# plt.subplot(255)
# plt.imshow(y[idx].numpy().argmax(-1),cmap='gray')
# plt.axis('off')
# plt.tight_layout()

# Move savefile to drive